In [1]:
import pandas as pd
import geopandas as gpd

/home/maciek/.pyenv/versions/3.8.12/envs/school-map-project/lib/python3.8/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [3]:
umwelt = gpd.read_file('../raw_data/Maps/umwelt.shp')
building_age = gpd.read_file('../raw_data/Maps/building_age.shp')

In [4]:
umwelt

,plr_name,laerm,luft,gruen,bio,status,anz_bel,wohnlage,geometry
0,Stülerstr.,mittel,hoch,"schlecht, sehr schlecht",hoch,mittel,3.0,Anteil der Adressen mit einfacher Wohnlage kle...,"POLYGON ((387256.566 5818552.267, 387323.142 5..."
1,Großer Tiergarten,None,hoch,"gut, sehr gut",mittel,None,1.0,Anteil der Adressen mit einfacher Wohnlage kle...,"POLYGON ((386767.495 5819393.112, 386768.323 5..."
2,Lützowstr.,mittel,hoch,"schlecht, sehr schlecht",hoch,mittel,3.0,Anteil der Adressen mit einfacher Wohnlage kle...,"POLYGON ((387952.632 5818274.543, 387986.668 5..."
3,Körnerstr.,mittel,hoch,mittel,hoch,niedrig,3.0,Anteil der Adressen mit einfacher Wohnlage kle...,"POLYGON ((388847.061 5817874.656, 388855.531 5..."
4,Nördlicher Landwehrkanal,mittel,hoch,"gut, sehr gut",hoch,hoch,2.0,Anteil der Adressen mit einfacher Wohnlage kle...,"POLYGON ((388129.510 5819015.126, 388157.136 5..."
...,...,...,...,...,...,...,...,...,...
442,Kafkastr.,niedrig - sehr niedrig,gering,"gut, sehr gut",mittel,hoch,0.0,Anteil der Adressen mit einfacher Wohnlage kle...,"POLYGON ((371696.804 5813210.407, 371820.892 5..."
443,Wannsee,mittel,gering,"gut, sehr gut",gering,hoch,0.0,Anteil der Adressen mit einfacher Wohnlage kle...,"POLYGON ((372422.918 5811591.863, 372493.826 5..."
444,Düppel,mittel,gering,"gut, sehr gut",mittel,hoch,0.0,Anteil der Adressen mit einfacher Wohnlage kle...,"POLYGON ((376253.232 5809145.865, 376274.142 5..."
445,Nikolassee,mittel,gering,"gut, sehr gut",gering,hoch,0.0,Anteil der Adressen mit einfacher Wohnlage kle...,"POLYGON ((375091.670 5812923.083, 375336.987 5..."
